In [ ]:
import os

from googleapiclient.discovery import build
from datetime import datetime, timedelta
import json
import numpy as np
import arrow
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches

from source.utils import authenticate

In [ ]:
data_dir = '../data/'

In [ ]:
save_fig = False
download_all_events = False

In [ ]:
creds = authenticate(verbose=True)
service = build('calendar', 'v3', credentials=creds)

In [ ]:
sns.set_style('whitegrid')

### Calendars 

In [ ]:
clist = service.calendarList().list().execute()

In [ ]:
def print_citem(citem):
    print('Summary:\t' + citem['summary'])
    print('id:\t\t' + citem['id'])
    print('***********')

In [ ]:
for citem in clist['items']:
    print_citem(citem)

#### Primary calendar 

In [ ]:
def print_default_calendar(clist):
    for citem in clist['items']:
        if citem.get('primary', False):
            print_citem(citem)

In [ ]:
print_default_calendar(clist)

### Colors 

In [ ]:
event_colors = service.colors().get().execute()['event']

In [ ]:
event_colors

In [ ]:
colors = {k: v['background'] for k, v in event_colors.items()}
colors

In [ ]:
fig, ax = plt.subplots(figsize=(12, 2))
for i, (num, color) in enumerate(colors.items()):
    rect = mpatches.Rectangle((i, 0), 1, 1, facecolor=color)
    ax.add_patch(rect)
    ax.text(i + 0.5, 0.5, num, ha='center', va='center', fontsize=12, fontweight='bold')

ax.set_xlim(0, len(colors))
ax.set_ylim(0, 1)
ax.axis('off')
plt.tight_layout()
plt.show()



In [ ]:
def_col = 11

In [ ]:
keyfile = data_dir + 'col2meaning.json'
with open(keyfile, 'r') as fo:
    col2meaning = json.load(fo)
col2meaning

In [ ]:
def get_event_type(etype: str, year:str = "default", col2meaning=col2meaning):
    if not isinstance(etype, str):
        etype = str(etype)
    if not isinstance(year, str):
        year = str(year)

    if year in col2meaning and etype in col2meaning[year]:
        return col2meaning[year][etype]
    elif etype in col2meaning["default"]:
        return col2meaning["default"][etype]
    else:
        return "unknown"
    

In [ ]:
types = list(col2meaning["default"].values())
# if 'default' in types:
#     types.remove('default')
types

### Events

In [ ]:
start = datetime(2014,1,1)
end = datetime(start.year + 1, 1,1)
now = arrow.get(datetime.now().isoformat() + '+02:00').datetime

end = now

start, end, now

In [ ]:
def print_event(e):
    print('Summary:\t' + e.get('summary', 'summary'))
    print('ColorId:\t' + e.get('colorId', 'unknown'))
    print('Start:\t\t' + str(e.get('start', 'start')))
    print('End:\t\t' + str(e.get('end', 'end')))
    print('Status:\t\t' + e.get('status', 'status'))
    print('====')

In [ ]:
def prepare_req(min_ts = start.isoformat() + 'Z', max_ts = end.isoformat() + 'Z'):

    print('get everything since', min_ts, 'until', max_ts)

    req_orig = service.events().list(calendarId='primary',
                                          timeMin=min_ts,
                                          #timeMax=max_ts,
                                          #maxResults=15, 
                                          singleEvents=True,
                                          orderBy='startTime')
    return req_orig


In [ ]:
req_orig = prepare_req()

In [ ]:
'''
col2meaning = { 
    e.get('colorId'): e.get('summary') for e in events[:-3]
}

col2meaning
'''
pass

In [ ]:
def print_event_res(events_result):
    print(f'got {len(events_result.get("items", []))} results')
    print('next page token:  ' + events_result.get('nextPageToken', 'no next page'))

In [ ]:
%%time

events_result = req_orig.execute()
events = events_result.get('items', [])
print_event(events[0])

print_event_res(events_result)

In [ ]:
def parse_to_datetime(time_obj):
    if type(time_obj) == datetime or type(time_obj) == 'float':
        return time_obj
    
    if 'date' in time_obj:
        s = time_obj['date']
    elif 'dateTime' in time_obj:
        s = time_obj['dateTime']
    else:
        s = time_obj
        
    return arrow.get(s).datetime

In [ ]:
def download_events(verbose=False):
    events = []
    prev_req = req_orig = prepare_req()
    events_result = req_orig.execute()

    events.append(events_result.get('items', []))
    prev_res = events_result

    i = 0
    while prev_req is not None:
        print(f'round {i}', end='\r')
        prev_req = service.events().list_next(prev_req, prev_res)
        if prev_req is None:
            break
        prev_res = prev_req.execute()
        
        if verbose: 
            print_event_res(prev_res)

        res_events = prev_res.get('items', [])

        events.append(res_events)

        last_start = parse_to_datetime(res_events[-1].get('start'))
        if last_start > now:
            print('last start was in the future, we can stop', last_start)
            break
        i += 1

    print('Finished fetching all the events')
    
    return events

In [ ]:
def parse_event(e):
    start = e.get('start')
    col = e.get('colorId', '0')
    end = e.get('end')
    summary = e.get('summary', '')

    whole_day = 'date' in start
    
    start = parse_to_datetime(start)
    end = parse_to_datetime(end)

    duration_s = (end - start).total_seconds() # <-- total seconds would compute also the full day events
    t = col2meaning[col]

    return {
        'start': start,
        'type': t,
        'summary': summary,
        'duration_s': duration_s,
        'whole_day': whole_day,
    }

In [ ]:
def group_datetime_by_week(ts):
    year, week = ts.isocalendar()[:2]
    return datetime.strptime(f'{year}-{min(week*7, 365)}', "%Y-%j")

In [ ]:
def get_quarter(ts):
    mon = ts.month
    quartal = mon // 3
    quartal += 1 if mon % 3 != 0 else 0 
    return quartal

def get_invoiced_quarter(ts):
    if type(ts) is float:
        return 666 # ts was not defined
    
    assert type(ts) is str, 'expected timestamp to be a string'
    mon = int(ts[5:7])
    quartal = mon // 3
    quartal += 1 if mon % 3 != 0 else 0 
    return quartal

### Download and save the events 

In [ ]:
def save_events(df, time_str):
    df.to_csv(data_dir + f'calendar_events_until_{time_str}.csv', index=False)

In [ ]:
%%time

if download_all_events:
    events = download_events()
    # includes all recurring future events as well
    events = np.concatenate(events) if type(events) is list else events
    print('events.shape', events.shape)
    
    print('available fields')
    print(events[0].keys())
    print()
    
    print_event(events[0])
    print_event(events[-1])

In [ ]:
if download_all_events:
    print('parsing events...')
    events_parsed = [ parse_event(e) for e in events ]
    
    events_all = pd.DataFrame(data=events_parsed)

    yesterday = now - timedelta(days=1)
    print(yesterday)
    
    hist_df = events_all[events_all.start < yesterday]

    save_events(hist_df, yesterday.strftime("%Y-%m-%d_%H:%M:%S"))
    print('hist df saved')

### Load the events from disk 

In [ ]:
def get_files_in_dir(directory, return_dirs=False, verbose=False):
    if not os.path.exists(directory):
        raise Exception(f'{directory} does not exist!')
    for (path, dirs, files) in os.walk(directory):
        if verbose:
            print('path: ', path)
            print('dirs', dirs)
            print('files')
            for i, file in enumerate(files):
                print('\t', i, file)
        break
    return files if not return_dirs else (files, dirs)

In [ ]:
files = get_files_in_dir(data_dir)
files

In [ ]:
ind = -1
assert files[ind].split('.')[-1] in 'csv', f'file was not a csv file! it was {files[ind]}'

events_file = data_dir + files[ind]
print(f'load events from file {events_file}')

with open(events_file, 'rb') as fo:
    events_all = pd.read_csv(events_file)
    
events_all

In [ ]:
events_all.groupby("type").agg({'summary': lambda x: x.unique()[-10:]}).reset_index().sort_values('type').to_numpy()

#### Durations 

In [ ]:
def process_durations(df):
    df['duration_min'] = df['duration_s'] / 60
    df['duration_h'] = df['duration_min'] / 60
    df['duration_d'] = df['duration_h'] / 24
    df['is_full_day'] = (df['duration_d'] >= 1) & (df['duration_h'] % 24 - 0 < 1e-6)
    df['whole_day'] = df['whole_day'] | df['is_full_day'] # this adds couple outlier events
    df['is_over_24h'] = df['duration_h'] > 24
    
    return df

In [ ]:
events_all = process_durations(events_all)

In [ ]:
events_all[~events_all.is_full_day & events_all.whole_day] # should be empty

In [ ]:
events_all[events_all.duration_d >= 1].sort_values('duration_d', ascending=False).head(10)

#### Dates 

In [ ]:
def process_dates(df):
    df['start'] = df.start.apply(lambda ts: parse_to_datetime(ts))
    df['year'] = df.start.apply(lambda ts: ts.year) #datetime.strptime(f'{ts.year}-1-1', '%Y-%m-%d'))
    df['year_mon'] = df.start.apply(lambda ts: datetime.strptime(f'{ts.year}-{ts.month}-1', '%Y-%m-%d'))
    df['mon'] = df.start.apply(lambda ts: datetime.strptime(f'2000-{ts.month}-1', '%Y-%m-%d'))
    df['year_week'] = df.start.apply(group_datetime_by_week)
    df['quarter'] = df.start.apply(get_quarter)
    return df

In [ ]:
events_all = process_dates(events_all)

In [ ]:
max_year = events_all.year.max()
print('max year', max_year)

In [ ]:
events = events_all[(events_all.whole_day == False) & (events_all.is_over_24h == False)]
events.head(10)

In [ ]:
def fill_in_missing_types(data, time_col='year_week', unique_times=None, col2meaning=col2meaning):
    """
    Fill in missing type entries for each time period with zero duration.
    
    Ensures complete type coverage across all time periods by adding entries with 
    duration_h=0 for any type-time combinations that don't exist in the data.
    
    Args:
        data (pd.DataFrame): DataFrame with time-series data containing type and duration columns
        time_col (str): Name of the time column (default: 'year_week')
        unique_times (array-like, optional): Specific time periods to process. If None, uses all unique times from data
        col2meaning (dict): Mapping of years (or "default") to expected type values
    
    Returns:
        pd.DataFrame: Original data with missing type entries added (duration_h=0), sorted by time and type
    """
    unique_times = unique_times if unique_times is not None else data[time_col].unique()
    
    to_append = []
    for week in unique_times:
        entries = data[data[time_col] == week]
        year = str(week.year) if type(week) is datetime else False
        types = np.array(list(col2meaning[year].values()) if year and year in col2meaning else list(col2meaning["default"].values()))

        types_in_week = set(entries['type'].astype(str))

        for t in set(types):
            if t not in types_in_week:
                to_append.append({'type': t, time_col: week, 'duration_h': 0})



    data = pd.concat([data, pd.DataFrame(to_append)], ignore_index=True)
    data = data.sort_values([time_col, 'type'])
        
    return data

In [ ]:
list(col2meaning.keys())

In [ ]:
if "type_ind" not in events.columns:
    events.loc[:, 'type_ind'] = events.type

type_year = []
# add all number & year combinations
for i, row in events.groupby(["type_ind", "year"]).agg({"duration_h": "sum"}).reset_index().iterrows():
    type_year.append((row.type_ind, row.year))

# for each combination, assign the proper type name
for type_ind, year in type_year:
    etype = get_event_type(type_ind, year)
    # print(f'type_ind: {type_ind}, year: {year} --> {etype}')
    mask = (events.type_ind == type_ind) & (events.year == year)
    events.loc[mask, 'type'] = etype
    # if len(events.loc[mask]) > 5:
    #     display(events.loc[mask, ["start", "type", "summary", "type_ind"]].sample(5))
    
events.sample(3)

In [ ]:
events_year = events.groupby(['type', 'year']).agg({'duration_h': "sum"}).reset_index()
events_year = fill_in_missing_types(events_year, 'year', col2meaning=col2meaning)
events_year = events_year.sort_values(['year', 'type'])
events_year

In [ ]:
events_week = events.groupby(['type', 'year_week']).agg({'duration_h': "sum"}).reset_index()
events_week = fill_in_missing_types(events_week, 'year_week', events_all.year_week.unique())
events_week['year'] = events_week.year_week.apply(lambda ts: ts.year)
events_week = events_week.sort_values(['year_week', 'type'])
events_week

### Event types 

In [ ]:
event_type_yearly = events.groupby(['type', 'year']).agg({'summary': lambda x: x.unique()[-5:]}).reset_index().to_numpy()
for row in event_type_yearly:
    etype, year, summaries = row
    print(f'Event type: {etype}')
    print(f"\tyear: {year}")
    for s in summaries:
        print('\t', s)
    print()

In [ ]:
event_colors['0'] = dict(background='blue')
event_colors

In [ ]:
year2palette = {}
for year in col2meaning:
    palette = {
        get_event_type(etype=c, year=year): event_colors[c]['background'] for c in event_colors
    }
    year2palette[year] = palette
year2palette

In [ ]:
def show_values_on_bars(axs):
    def _show_on_single_plot(ax):        
        for p in ax.patches:
            _x = p.get_x() + p.get_width() / 2
            _y = p.get_y() + p.get_height()
            value = str(int(round(p.get_height(),0)))
            ax.text(_x, _y, value, ha="center", fontsize=8) 

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(12, 8), dpi=120, facecolor="w")

data = events.groupby("type").agg({"duration_d": np.sum}).reset_index()

sns.barplot(
    data=data.sort_values("type"), x="type", y="duration_d", hue="type", ax=axes, palette=palette
)
show_values_on_bars(axes)
plt.title(f"Use of time per event type from 2015-{max_year}")
plt.ylabel("Total duration (days)")
plt.xticks(rotation=90)
plt.xticks(axes.get_xticks())

plt.show()

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(12,8), dpi=120, facecolor='w')

sns.barplot(data=events.sort_values('type'), x='type', y='duration_h', hue='type', ax=axes, palette=palette)
plt.title('Distribution of event duration')
plt.ylabel("Total duration (hours)")
plt.xticks(rotation=90)
plt.xlabel('Event type')

plt.show()

### Event types over time 

#### By year 

In [ ]:
time_agg = 'duration_h'
data = events.groupby(['type', 'year']).agg({time_agg: "sum"}).reset_index()
data = data.sort_values('year')

n_years = data.year.nunique()

top = 1.05 * data[time_agg].max()

fig, axes = plt.subplots(n_years,1,figsize=(12,6*n_years), dpi=120, facecolor='w')

for i, year in enumerate(data.year.unique()):
    ax = axes[i]
    
    query = f'year == {year}'
    sns.barplot(data=data.query(query).sort_values('type'), x='type', y=time_agg, ax=ax, palette=palette, hue='type')
    ax.set_title(f'{year}')
    ax.set_ylabel(f'Total duration ({time_agg.split("_")[-1]})')
    
    show_values_on_bars(ax)
    ax.set_ylim(top=top)
    ax.xaxis.set_tick_params(rotation=90)

plt.tight_layout()
plt.show()

In [ ]:
default_events = events[events['type'] == "default"]
for year in default_events.year.unique():
    print(f'Year: {year}')
    summaries = default_events[default_events.year == year]['summary'].sample(10)
    for s in summaries:
        print('\t', s)
    print()

#### By week 

In [ ]:
year2palette.keys()

In [ ]:
palette

In [ ]:
time_agg = 'duration_h'
data = events_week
n_years = data.year.nunique()
print(f"visualise {n_years} years of data: {data.year.unique()}")

# data = fill_in_missing_types(data, 'year_week')

top = 1.05 * data[time_agg].max()

fig, axes = plt.subplots(n_years,1,figsize=(12,6*n_years), dpi=120, facecolor='w')

for i, year in enumerate(data.year.unique()):
    ax = axes[i]
    print(f'Processing year {year}...')
    query = f'year == {year}'
    pal = year2palette.get(str(year), palette)
    data_year = data.query(query).sort_values('type')
    type_replacaments = {}
    for etype in data_year['type'].unique():
        if etype not in pal:
            print(f"\tWarning: type '{etype}' not found in palette for year {year}")
            new_type = col2meaning.get(str(year), col2meaning["default"]).get(etype, 'unknown')
            type_replacaments[etype] = new_type
    if type_replacaments:
        print(f'\tReplacing types: {type_replacaments}')
        data_year['type'] = data_year['type'].replace(type_replacaments)

    sns.lineplot(data=data_year, x='year_week', y=time_agg, hue='type', ax=ax, palette=pal, estimator=None)
    ax.set_title(f'{year}')
    
    ax.set_ylim(top=top)

plt.tight_layout()
plt.show()

In [ ]:
for year in col2meaning:
    print(f'Year: {year}')
    etypes = col2meaning[year].values()
    for i, t in enumerate(etypes):
        print(i, t)
    print()

In [ ]:
work_types = [types[5], types[7], types[9]]
work_types

In [ ]:
time_agg = 'duration_h'

any_work_hours = np.array([data[data.type == t][time_agg] for t in work_types]).sum(axis=0)
any_work_hours[-10:]

In [ ]:
data = events_week

fig, axes = plt.subplots(1,1,figsize=(18,8), dpi=120, facecolor='w')

ax = axes


query = f'type == "{types[0]}"'
sns.lineplot(x=data.year_week.unique(), y=data.query(query)[time_agg], ax=ax, label=types[0])

sns.lineplot(x=data.year_week.unique(), y=any_work_hours, ax=ax, label='efficient working hours')

ax.axhline(37.5, ls='--', color='k', zorder=0, label='full work week')

plt.legend()

plt.title('Efficient working hours per week')
plt.tight_layout()
plt.show()

In [ ]:
working_df = pd.DataFrame(data={'year_week': data.year_week.unique(), 'working_h': any_work_hours})
working_df

In [ ]:
working_df['overtime'] = working_df.working_h.apply(lambda x: x >= 37.5)
working_df['clear overtime'] = working_df.working_h.apply(lambda x: x >= 40)
working_df

### Busiest weeks 

In [ ]:
working_df_sorted = working_df.sort_values(['working_h', 'year_week'], ascending=[False, True]).reset_index()

In [ ]:
working_df_sorted[:10]

In [ ]:
# Pivot the data to create a matrix for the heatmap
heatmap_data_month = working_df.pivot_table(
    index=working_df["year_week"].dt.year,
    columns=working_df["year_week"].dt.month,
    values="working_h",
    aggfunc="sum",
    fill_value=0,
).rename_axis("Month", axis=1).rename_axis("Year", axis=0)

# Drop rows where the sum of the row is zero
heatmap_data_month.drop(
    heatmap_data_month.sum(axis=1)[heatmap_data_month.sum(axis=1) == 0].index, inplace=True
)

heatmap_data_month = heatmap_data_month / 7.5
heatmap_data_month

In [ ]:
# Create the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data_month, cmap="coolwarm", cbar_kws={'label': 'Working Days (PWD); 21 days is a full working month'}, annot=True, fmt=".0f", center=21)
plt.title("Heatmap of Work Time (PWD) by Month")
plt.xlabel("Month")
plt.ylabel("Year")
plt.tight_layout()
plt.show()

In [ ]:
# Pivot the data to create a matrix for the heatmap
heatmap_data_week = working_df.pivot_table(
    index=working_df["year_week"].dt.year,
    columns=working_df["year_week"].dt.isocalendar().week,
    values="working_h",
    aggfunc="sum",
    fill_value=0,
).rename_axis("Week", axis=1).rename_axis("Year", axis=0)

# remove w53
heatmap_data_week.loc[:, 52] += heatmap_data_week.loc[:, 53]
heatmap_data_week.drop(53, inplace=True, axis=1)

# remove years before 2018
heatmap_data_week = heatmap_data_week.loc[2018:]

# Drop rows where the sum of the row is zero
heatmap_data_week.drop(
    heatmap_data_week.sum(axis=1)[heatmap_data_week.sum(axis=1) == 0].index, inplace=True
)

heatmap_data_week = heatmap_data_week / 7.5
heatmap_data_week

In [ ]:
# Pivot the data to create a matrix for the heatmap
heatmap_data_overtime = working_df.pivot_table(
    index=working_df["year_week"].dt.year,
    columns=working_df["year_week"].dt.isocalendar().week,
    values="overtime",
    # aggfunc="sum",
    fill_value=0,
).rename_axis("Week", axis=1).rename_axis("Year", axis=0)

heatmap_data_overtime.drop(53, inplace=True, axis=1)

# # remove years before 2018
heatmap_data_overtime = heatmap_data_overtime.loc[2018:]

heatmap_data_overtime

In [ ]:
# Create the heatmap
plt.figure(figsize=(15, 8))
how_much_overtime_week = (heatmap_data_week-5).where(heatmap_data_week > 5, other=0)
sns.heatmap(heatmap_data_overtime, cmap="coolwarm", cbar_kws={'label': 'Working Days (PWD); 5 days is a full work week'}, annot=how_much_overtime_week, fmt=".0f")
plt.title("Heatmap of Weeks with Overtime (PWD) by Week")
plt.xlabel("Week")
plt.ylabel("Year")
plt.tight_layout()
plt.show()

In [ ]:
# Create the heatmap
plt.figure(figsize=(15, 8))
sns.heatmap(how_much_overtime_week, cmap="coolwarm", cbar_kws={'label': 'Working Days (PWD); 5 days is a full work week'}, annot=True, fmt=".0f", center=1)
plt.title("Heatmap of Weeks with Overtime (PWD) by Week")
plt.xlabel("Week")
plt.ylabel("Year")
plt.tight_layout()
plt.show()

### Number of overtime days per year

In [ ]:
working_df.info()

In [ ]:
working_df["year"] = working_df.year_week.apply(lambda ts: ts.year)
working_df["quarter"] = working_df.year_week.apply(lambda ts: get_quarter(ts))
working_df["year_quarter"] = working_df.year_week.apply(lambda ts: f"{ts.year}-Q{get_quarter(ts)}")
working_df.tail(2)

In [ ]:
df_working = working_df.copy().drop(columns=['overtime', 'clear overtime'])
df_working.loc[:, "overtime"] = working_df.overtime.apply(lambda x: 'overtime' if x else 'no overtime')
df_working.loc[:, "clear overtime"] = working_df["clear overtime"].apply(lambda x: 'clear overtime' if x else 'no overtime')
df_working.info()

In [ ]:
y_col = "overtime"
df_overtime_yearly = df_working.groupby([y_col, 'year']).nunique()['year_week'].reset_index().sort_values('year')
df_overtime_yearly.rename(columns={'year_week': 'n_weeks'}, inplace=True)
df_overtime_quarterly = df_working.groupby([y_col, 'year_quarter']).nunique()['year_week'].reset_index().sort_values('year_quarter')
df_overtime_quarterly.rename(columns={'year_week': 'n_weeks'}, inplace=True)
df_overtime_yearly

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(12,6), dpi=120, facecolor='w')
ax = axes[0]
sns.barplot(data=df_overtime_yearly, x='year', y='n_weeks', hue=y_col, ax=ax)
ax.axhline(47/2, ls='--', color='black', zorder=0, label='every other work week is overtime')
ax.legend()
ax.set_title('Overtime weeks per year')

ax = axes[1]
sns.barplot(data=df_overtime_quarterly, x='year_quarter', y='n_weeks', hue=y_col, ax=ax)
ax.set_title('Overtime weeks per quarter')
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
ax.set_xlabel('Year-Quarter')
ax.axhline(52/4/2, ls='--', color='black', zorder=0, label='every other work week is overtime')

for ax in axes:
    ax.set_ylabel('Number of weeks')

plt.legend()
plt.tight_layout()
plt.show()